In [400]:
import pandas as pd

In [401]:
class Quiz:
    def __init__(self, nome_quiz, total_questoes):
        self.nome_quiz = nome_quiz
        self.total_questoes = total_questoes
        self.acertos = 0
        self.erros = 0
        self.duracao_total = 0
        self.user = ''
        self.tentativa = 0
        self.resultados = {}

    
    # Função responsável por contabilizar os acertos, os erros e o tempo toda da partida
    def respostas (self, user='',  tentativa=0, resposta_correta=0, tempo_resposta=0):

        self.tentativa = tentativa

        if self.tentativa > 0:
            self.user = user
            self.duracao_total += tempo_resposta

            if resposta_correta:
                self.acertos += 1
            else:
                self.erros += 1
                
            self.resultados[self.user,f"Tentativa {self.tentativa}"]= {
                    'Quiz':             self.nome_quiz,
                    'Acertos':          self.acertos,
                    'Erros':            self.erros,
                    'Duração total':    self.duracao_total
                }
        data_resutados = pd.DataFrame(self.resultados).T
        return data_resutados
        
    def avaliar_desempenho(self):

        # Calculando desempenho
        media_tempo_resposta = self.duracao_total / self.total_questoes
        percentual_acertos = (self.acertos / self.total_questoes) * 100

        # Avaliando o resultado do usuário
        if percentual_acertos >= 70 and media_tempo_resposta <= 20:
            mensagem = "APROVADO: Você atingiu o desempenho necessário para passar de fase."
        else:
            mensagem = "REPROVADO: Você não atingiu o desempenho necessário para passar de fase."

        # Resetando variáveis
        self.acertos = 0
        self.erros = 0
        self.duracao_total = 0
        
        return self.nome_quiz, self.tentativa, percentual_acertos, media_tempo_resposta, mensagem



In [402]:
# INPUT INICIAL
nome_quiz       = 'Exame - hemograma'
total_questoes  = 10

quiz = Quiz(nome_quiz,total_questoes)

# Analise de cada tentativa

In [403]:
def analise_tentativa (user, data, tentativa):
    data_filtrado =  data.loc[(data['User_ID'] == user) & (data['Tentativa'] == f'Tentativa_{tentativa}'), 'Respostas'].iloc[0]
    desempenho_por_tentativa(user, data_filtrado, tentativa)


def passando_dados (user_id, respostas_usuario, tentativa):

        for questao in range(1, total_questoes):
                respostas   = respostas_usuario[questao][0]
                tempo       = respostas_usuario[questao][1]
                        
                quiz.respostas(user_id, tentativa, respostas, tempo)



def desempenho_por_tentativa (user_id, respostas_usuario, tentativa):

        # Passando os dados
        passando_dados(user_id, respostas_usuario, tentativa)

        # Buscando as informações da função avaliar_desempenho
        quiz_nome, tentativa, percentual_acertos, media_tempo_resposta, resultado = quiz.avaliar_desempenho()

        print(f"""
                QUIZ: {quiz_nome}

                Usuário .............................. {user_id}
                Tentativa realizada .................. {tentativa}° tentativa
                Percentual de acertos ................ {percentual_acertos:.2f}%
                Média do tempo de resposta ........... {media_tempo_resposta:.2f} segundos

                {resultado}
                """)
        

# Analise do progresso

In [404]:
def progresso (user):
    data_user = quiz.respostas().loc[user]
    user = 'User_1'

    df = quiz.respostas().loc[user]

    tentativas = len(df)

# Gerando DataFrame teste

In [405]:
import pandas as pd
import random

# Função para gerar tentativas aleatórias
def gerar_tentativa():
    tentativa = []
    for _ in range(10):  # 10 questões no quiz
        resultado = random.choice([True, False])  # Escolhe aleatoriamente se a resposta está correta ou não
        tempo = random.randint(10, 30)  # Tempo aleatório entre 10 e 30 segundos
        tentativa.append((resultado, tempo))
    return pd.Series(tentativa)

# Dicionário para armazenar as respostas dos usuários
respostas = {}

# Gerar 10 novos usuários com 4 tentativas diferentes cada
for i in range(1, 11):
    user_id = f'User_{i}'
    tentativas_usuario = {}
    for j in range(1, 5):
        nome_tentativa = f'Tentativa_{j}'
        tentativa = gerar_tentativa()
        tentativas_usuario[nome_tentativa] = tentativa
    respostas[user_id] = tentativas_usuario

# Criar DataFrame com os resultados das tentativas dos usuários
data = []
for user_id, tentativas_usuario in respostas.items():
    for tentativa, respostas_tentativa in tentativas_usuario.items():
        data.append([user_id, tentativa, respostas_tentativa])

# Criar DataFrame a partir dos dados coletados
df = pd.DataFrame(data, columns=['User_ID', 'Tentativa', 'Respostas'])

# Mostrar DataFrame
display(df.head(5))

,User_ID,Tentativa,Respostas
0,User_1,Tentativa_1,"0 (True, 26) 1 (True, 22) 2 (True,..."
1,User_1,Tentativa_2,"0 (False, 11) 1 (False, 30) 2 (True,..."
2,User_1,Tentativa_3,"0 (False, 23) 1 (False, 24) 2 (False,..."
3,User_1,Tentativa_4,"0 (True, 25) 1 (True, 30) 2 (True,..."
4,User_2,Tentativa_1,"0 (False, 30) 1 (False, 14) 2 (True,..."


In [406]:
analise_tentativa('User_1',df, 1)
analise_tentativa('User_1',df, 2)


                QUIZ: Exame - hemograma

                Usuário .............................. User_1
                Tentativa realizada .................. 1° tentativa
                Percentual de acertos ................ 50.00%
                Média do tempo de resposta ........... 16.10 segundos

                REPROVADO: Você não atingiu o desempenho necessário para passar de fase.
                

                QUIZ: Exame - hemograma

                Usuário .............................. User_1
                Tentativa realizada .................. 2° tentativa
                Percentual de acertos ................ 50.00%
                Média do tempo de resposta ........... 15.60 segundos

                REPROVADO: Você não atingiu o desempenho necessário para passar de fase.
                


# Testes

In [407]:
user ='User_1'
data_filtrado =  df.loc[(df['User_ID'] == user)]
data_filtrado


,User_ID,Tentativa,Respostas
0,User_1,Tentativa_1,"0 (True, 26) 1 (True, 22) 2 (True,..."
1,User_1,Tentativa_2,"0 (False, 11) 1 (False, 30) 2 (True,..."
2,User_1,Tentativa_3,"0 (False, 23) 1 (False, 24) 2 (False,..."
3,User_1,Tentativa_4,"0 (True, 25) 1 (True, 30) 2 (True,..."
